# Úkol č. 1 - vizualizace dat a web scraping (do 11. října)

  * V rámci tohoto úkolu musíte stáhnout dat z webu (tzv. _web scraping_, velmi základní) a následně data zpracovat a vizualizovat.
  * Cílem bude stáhnout data ze serveru https://www.psp.cz/sqw/hlasovani.sqw?o=8 týkající hlasování současné poslanecké sněmovny, uložit tato data v tabulkovém formátu a pak vymyslet vizualizace a zobrazení dat, které umožní orientaci v těchto datech a zvýrazní zajímavé informace a zobrazit přehledně časový vývoj různých veličin.
 
> **Úkoly jsou zadány tak, aby Vám daly prostor pro invenci. Vymyslet _jak přesně_ budete úkol řešit, je důležitou součástí zadání a originalita či nápaditost bude také hodnocena!**

## Data

 * Měli byste stáhnout data ze všech hlasování současné poslanecké sněmovny (tj. od voleb v roce 2017) a to až na úroveň jednotlivých poslanců. 
 * Data by měla obsahovat i stručný popis toho, o jaké hlasování šlo.

## Pokyny k vypracování

**Základní body zadání**, za jejichž (poctivé) vypracování získáte **8 bodů**:
  * Strojově stáhněte data a uložte je do vhodného formátu, který se Vám bude dále dobře zpracovávat. 
  * **I když nemusíte mít sněmovnu rádi, snažte se nezahltit server a mezi požadavky na stránku mějte alespoň vteřinu mezeru.**
  * V druhé části Vašeho Jupyter notebooku pracujte se staženými daty v souborech, které jsou výsledkem Vašeho stahovacího skriptu (aby opravující nemusel spouštět stahování z webu).
  * S využitím vybraných nástrojů zpracujte data a vymyslete vizualizace a grafy, aby bylo vidět následující:
    * Odchody a přeběhnutí poslanců mezi jednotlivými stranami.
    * Jakou mají jednotliví poslanci docházku (účast na hlasování) a jak jsou na tom s docházkou strany jako celek.
    * Jak často spolu jednotlivé strany hlasují ve shodě a jak často hlasují odlišně.
    * Jak jsou poslanci jednotlivých stran jednotní v hlasování (jak často hlasují stejně, kdo jsou největší rebelové).

**Další body zadání** za případné další body (můžete si vybrat, maximum bodů za úkol je každopádně 12 bodů):
  * Ve vizualizacích a grafech nějak zachyťte časový vývoj (např. jak se mění docházka, shoda mezi stranami atp.).
  * Najděte jednotlivé poslance, kteří se nejvíce shodují při svém hlasování či kteří mají co nejpodobnější účast na hlasování.
  * Zkuste rozlišit v datech důležitá hlasování a zpracujte vizualizace (také) pro ně.
  * Zkuste najít hlasování, kde poslanci hlasovali nejvíce jinak, než obvykle.
  
## Tipy a triky
  * Balíčky iportujte na začátku notebooku (příp. na začátku scrapovací a pak na začátku vizualizační části).
  * Využívejte markdown buňky (jako je ta, ve které je tento text) a naspisy, abyste usnadnily orientaci opravující(mu).
  * Pečlivě si vybírejte grafy a vizualizace, aby co nejlépe vyjádřily to, co vyjádřit chcete. Doporučuji projít si pro inspiraci alespoň galerie balíčků `matplotlib` a `seaborn`.

## Poznámky k odevzdání

  * Řiďte se pokyny ze stránky https://courses.fit.cvut.cz/BI-VZD/homeworks/index.html.
  * Odevzdejte Jupyter Notebook (příp. doplněn skripty), ale i soubor(y) se staženými daty (ať opravující nemusí stahovat data).
  * Opravující Vám může umožnit úkol dodělat či opravit a získat tak další body. První verze je ale důležitá a bude-li odbytá, budete za to penalizováni.

In [154]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
url = " https://www.psp.cz/sqw/hlasovani.sqw?o=8"
page = requests.get(url)
soap = BeautifulSoup(page.text, "html.parser")
schuze_pre=soap.select("tbody > tr td.col-number a")
schuze=[]
for tag in schuze_pre:
    schuze.append(tag.text.split('.')[0])
print('all available schuze:\n',schuze)

all available schuze:
 ['1', '2', '3', '4', '5', '6', '7', '8', '9', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61']


In [187]:
import time

dif process_hlasovani(html_hlasovani, dataframes):
    pass

# determine amount of pages of hlasovani table for POST request
def amount_of_pages(html_first_schuze_page):
    soap = BeautifulSoup(html_first_schuze_page,'html.parser')
    selection = soap.select("div#main-content > div.search-results > p.paging > span.pages > a")
    if not selection:
        return 1
    elif selection[-1].text == '>>':
        return int(selection[-1]['href'].split('=')[-1])
    else:
        return int(selection[-2].text)

def process_schuze_page(html_schuze_page, dataframes):
    local_df = pd.read_html(html_schuze_page)[0]
    time.sleep(0.5)
    dataframes[0] = dataframes[0].append(local_df, ignore_index = True)

container =[
    pd.DataFrame(columns = ['Schůze','Č. hl.','Bod','Název','Datum','Výsledek']),
    pd.DataFrame()
]

schuze_post_data={
    'o': '8',
    's': '1',
    'pg': '1'
}
schuze_post_url='https://www.psp.cz/sqw/phlasa.sqw'

for s in schuze:
    schuze_post_data['s'] = s
    schuze_post_data['pg'] = '1'
    r = requests.post(schuze_post_url, schuze_post_data)
    npages = amount_of_pages(r.text)
    print('schuzka',s,': amount of pages:', npages)
    
    # process first page
    process_schuze_page(r.text, container)
    # process others
    for page in range(2, npages + 1):
        schuze_post_data['pg'] = str(page)
        r = requests.post(schuze_post_url, schuze_post_data)
        process_schuze_page(r.text, container)
    
    display(container[0])
    time.sleep(0.5)

display(schuze_df)

schuzka 1 : amount of pages: 1


,Schůze,Č. hl.,Bod,Název,Datum,Výsledek
0,1,1,3.0,Inf. o ustavení volební komise PS a volbě členů,20. 11. 2017 13:53,Přijato (zmatečné)
1,1,2,3.0,Inf. o ustavení volební komise PS a volbě členů,20. 11. 2017 13:53,Přijato
2,1,3,NaN,Procedurální hlasování,20. 11. 2017 14:47,Zamítnuto
3,1,4,4.0,Náv.na stanovení počtu čl. MIV,20. 11. 2017 15:39,Zamítnuto
4,1,5,4.0,Náv.na stanovení počtu čl. MIV,20. 11. 2017 15:39,Přijato
5,1,6,6.0,Návrh na ustavení MIV,22. 11. 2017 10:06,Přijato
6,1,7,7.0,Návrh na potvrzení předsedy MIV,22. 11. 2017 12:57,Přijato
7,1,8,8.0,Inf. MIV o ověření platnosti volby poslanců,22. 11. 2017 13:00,Přijato
8,1,9,10.0,Návrh na stanovení počtu místopředsedů PS,22. 11. 2017 17:23,Zamítnuto
9,1,10,10.0,Návrh na stanovení počtu místopředsedů PS,22. 11. 2017 17:24,Přijato


schuzka 2 : amount of pages: 4


,Schůze,Č. hl.,Bod,Název,Datum,Výsledek
0,1,1,3.0,Inf. o ustavení volební komise PS a volbě členů,20. 11. 2017 13:53,Přijato (zmatečné)
1,1,2,3.0,Inf. o ustavení volební komise PS a volbě členů,20. 11. 2017 13:53,Přijato
2,1,3,NaN,Procedurální hlasování,20. 11. 2017 14:47,Zamítnuto
3,1,4,4.0,Náv.na stanovení počtu čl. MIV,20. 11. 2017 15:39,Zamítnuto
4,1,5,4.0,Náv.na stanovení počtu čl. MIV,20. 11. 2017 15:39,Přijato
...,...,...,...,...,...,...
85,2,70,18.0,Náv. na potvrzení předsedů výborů PS,30. 11. 2017 9:31,Přijato
86,2,71,30.0,Návrh časového harmonogramu projednávání SR 2018,30. 11. 2017 9:38,Přijato
87,2,72,31.0,Žádost o vyslovení souhlasu s tr. stíháním pos...,30. 11. 2017 9:41,Přijato
88,2,73,32.0,Žádost o vyslovení souhlasu PS k trestnímu stí...,30. 11. 2017 9:44,Přijato


schuzka 3 : amount of pages: 1


...

,Schůze,Č. hl.,Bod,Název,Datum,Výsledek
0,1,1,3.0,Inf. o ustavení volební komise PS a volbě členů,20. 11. 2017 13:53,Přijato (zmatečné)
1,1,2,3.0,Inf. o ustavení volební komise PS a volbě členů,20. 11. 2017 13:53,Přijato
2,1,3,NaN,Procedurální hlasování,20. 11. 2017 14:47,Zamítnuto
3,1,4,4.0,Náv.na stanovení počtu čl. MIV,20. 11. 2017 15:39,Zamítnuto
4,1,5,4.0,Náv.na stanovení počtu čl. MIV,20. 11. 2017 15:39,Přijato
...,...,...,...,...,...,...
1019,17,3,1.0,Žádost vlády České republiky o vyslovení důvěry,11. 7. 2018 18:09,Přijato
1020,17,4,1.0,Žádost vlády České republiky o vyslovení důvěry,11. 7. 2018 21:43,Zamítnuto (zmatečné)
1021,17,5,1.0,Žádost vlády České republiky o vyslovení důvěry,11. 7. 2018 21:43,Zamítnuto
1022,17,6,1.0,Žádost vlády České republiky o vyslovení důvěry,11. 7. 2018 21:46,Zamítnuto


schuzka 18 : amount of pages: 1


,Schůze,Č. hl.,Bod,Název,Datum,Výsledek
0,1,1,3.0,Inf. o ustavení volební komise PS a volbě členů,20. 11. 2017 13:53,Přijato (zmatečné)
1,1,2,3.0,Inf. o ustavení volební komise PS a volbě členů,20. 11. 2017 13:53,Přijato
2,1,3,NaN,Procedurální hlasování,20. 11. 2017 14:47,Zamítnuto
3,1,4,4.0,Náv.na stanovení počtu čl. MIV,20. 11. 2017 15:39,Zamítnuto
4,1,5,4.0,Náv.na stanovení počtu čl. MIV,20. 11. 2017 15:39,Přijato
...,...,...,...,...,...,...
1032,18,9,2.0,Novela z. o platu představitelů státní moci (s...,22. 8. 2018 15:13,Zamítnuto
1033,18,10,2.0,Novela z. o platu představitelů státní moci (s...,22. 8. 2018 15:14,Přijato
1034,18,11,2.0,Novela z. o platu představitelů státní moci (s...,22. 8. 2018 15:15,Přijato
1035,18,12,2.0,Novela z. o platu představitelů státní moci (s...,22. 8. 2018 15:16,Zamítnuto


schuzka 19 : amount of pages: 13


KeyboardInterrupt: 